# Import Module

In [11]:
import requests
import re
from bs4 import BeautifulSoup
from time import sleep
from random import random

# User Input

In [12]:
def user_input():
    while True:
        try:
            movie_name = input("請輸入欲搜尋的電影名稱：")
            if movie_name.strip():
                return movie_name
        except:
            print('\nTry Again')
            return None

def search_movie(query):
    # search
    encoded_query = requests.utils.quote(query)
    url = f"https://movies.yahoo.com.tw/moviesearch_result.html?keyword={encoded_query}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    movie_titles = soup.find_all("div", class_="release_movie_name")

    # check
    def search_check(movie_titles):
        res_count = len(movie_titles)
        if res_count == 0:
            print(f'沒有找到{query}')
            return None
        elif res_count > 1:
            res = movie_titles[0]
            ans = None
            while ans != 'Y' and ans != 'N':
                ans = input(f'找到多部電影，返回第一個搜尋{res.a.text}(Y)或是重新搜尋(N):')

            if ans == 'Y':
                return res.a['href']
            elif ans == 'N':
                search_movie(user_input())
                sleep(1 + random())
        else:
            return movie_titles[0].a['href']
    
    return search_check(movie_titles)

# Get specific moive infomation

In [13]:
def specific_moive_info(movie_link):
    def review_latest(review_link):
        if review_link == None:
            return None
        res = []
        response = requests.get(review_link)
        soup = BeautifulSoup(response.text, "html.parser")
        reviews = soup.find_all('div', class_='usercom_inner _c')
        for i in reviews[:5]:
            # res.append(i.find_all('span')[-1].text.strip().replace('\r\n', ''))
            res.append(re.sub(r'[\s]{2,}|[\n\r\t]+','',i.find_all('span')[-1].text))
        return res

    # movie & reviewlink
    movie_info = {}
    response = requests.get(movie_link)
    soup = BeautifulSoup(response.text, "html.parser")
    movie = soup.find('div', class_ = 'movie_intro_info_r')
    review = soup.find('div', class_ ='btn_plus_more usercom_more gabtn')
    review_link = review.a['href'] if review else None
    starscore = soup.find('div', class_ = 'score_num count')
    starbox = soup.find('div', class_ = 'starbox2')

    # moiveinfo
    chname = re.sub(r'[\s]{2,}|[\n\r\t]+','',movie.h1.text)
    engname = re.sub(r'[\s]{2,}|[\n\r\t]+','',movie.h3.text) if movie.h3.text else None
    
    if starscore and starbox:
        starscore = f'{starscore.text} / 5 {starbox.text.strip()}'
    else:
        starscore = None
    
    reviews = review_latest(review_link)

    release_date, IMDB = None, None
    for i in movie.find_all('span'):
        text = i.text.strip()
        if '上映日期' in text:
            release_date = text.split('：')[1]
        elif 'IMDb分數' in text:
            IMDB = f"{text.split('：')[1]} / 10"
    movie_info['中文名稱'] = chname
    movie_info['英文名稱'] = engname
    movie_info['上映日期'] = release_date
    movie_info['滿意度'] = starscore
    movie_info['IMDb'] = IMDB
    movie_info['最新5則評論'] = reviews if reviews else None

    return movie_info

# Ouput

In [19]:
res = search_movie(user_input())
if res:
    movie_info = specific_moive_info(res)
    for k,v in movie_info.items():
        if k != '最新5則評論':
            print(k,v)
        else:
            if v:
                print(k,':')
                for idx, val in enumerate(v,1):
                    print(idx, val)
            else:
                print(k,v)

中文名稱 鋼鐵人3
英文名稱 IRON MAN 3
上映日期 2013-04-24
滿意度 4.5 / 5 (共3861人投票)
IMDb 7.2 / 10
最新5則評論 :
1 新鮮度有下降1.2集感覺比較好看一些
2 一二集好看多了.第三集根本只是看換裝秀.劇情和戰鬥場面都普通.我想看的是更多驚人的武器.漫畫裡有那麼多超炫的技能.電影都沒看到.越看越無聊
3 鋼鐵人好棒，小勞勃道尼實在魅力十足
4 不就鋼鐵人很屌?打來打去而已，沒什麼深度劇情，這種東西最多只能給3顆星，但是我不爽大家都那麼支持商業化的爽片，所以給一顆
5 看完鋼鐵人就是覺得好看 看完不會失望 好看^^
